In [150]:
import re
from glob import glob
from tqdm import tqdm_notebook as tqdm
from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

pd.set_option("display.max_rows", 300)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## noise analysis

In [2]:
trn_df = pd.read_csv('../inputs/nes_info/e080_dataset_trn_df.csv').dropna()
display(trn_df.shape, trn_df.head())

(27423, 12)

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",7,9
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,2,3
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,2,2
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,3,3
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",3,4


In [96]:
magic_trn_df = trn_df[trn_df.text.str.contains('    ')]
magic_trn_df.head()

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,is back home now gonna miss every one,onna,False,miss,gonna,onna,0,0
88,bffa3ddd61,i miss you bby wish you were going tomorr...,i miss you bby,negative,i miss you bby wish you were going tomorrow to...,i miss you bby,False,i miss,i miss you bby,i miss you bby,4,5
110,9c4817f73b,if u have a friendster add me!!!!!!!!! ...,if u have a friendster add me!!!!!!!!! ...,neutral,if u have a friendster add me!!!!!!!!! my emai...,if u have a friendster add me!!!!!!!!! my emai...,False,if u have a friendster add me!!!!!!!!! my ema...,if u have a friendster add me!!!!!!!!! my ema...,if u have a friendster add me!!!!!!!!! my ema...,12,22
226,151c10cc39,JONAS BROTHERS - Live to party. ...,JONAS BROTHERS - Live to party. ...,positive,JONAS BROTHERS - Live to party. It`s rocking s...,JONAS BROTHERS - Live to party. It`s rocking s...,False,"i love the song,",jonas brothers - live to party. it`s rocking ...,jonas brothers - live to party. it`s rocking ...,10,15
430,5010f92b03,Took a shift tomorrow. I don`t really feel ...,I don`t really feel like,negative,Took a shift tomorrow. I don`t really feel lik...,I don`t really feel like,False,i don`t really feel like working right now.,i don`t really feel like,i don`t really feel like,5,7


In [98]:
magic_trn_df.iloc[0]['text']

'is back home now      gonna miss every one'

In [11]:
def get_char_targets(src_text, selected_text):
    src_text = row.text
    selected_text = row.selected_text
    idx0, idx1 = None, None
    for i, char in enumerate(src_text):
        if char == selected_text[0] and src_text[i: i+len(selected_text)] == selected_text:
            idx0, idx1 = i, i + len(selected_text)
            break
    assert src_text[idx0:idx1] == selected_text

    char_targets = [0] * len(src_text)
    if idx0 is not None and idx1 is not None:
        for i in range(idx0, idx1):
            char_targets[i] = 1

    return char_targets

def preprocess(text):
    maxlen = len(text)
    text = ' '.join(text.strip().split())
    text += ' ' * (maxlen - len(text))
    return text

In [14]:
for i, row in magic_trn_df.sample(20, random_state=72).iterrows():
    print(row['selected_text'])
    with pd.option_context('display.max_columns', None):
        df = pd.DataFrame(dict(
            char_targets=get_char_targets(row.text, row.selected_text),
            raw_text=list(row.text),
            fixed_text=list(preprocess(row.text)),
        )).T
        display(df)

Thanks


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
char_targets,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
raw_text,T,h,a,n,k,s,,,,,_,T,w,i,t,0,1
fixed_text,T,h,a,n,k,s,,_,T,w,i,t,0,1,,,


I`m sa


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0
raw_text,H,a,d,,a,,c,r,a,z,y,,n,i,g,h,t,",",,l,o,s,t,,k,e,y,s,",",,w,a,l,k,e,d,,h,o,m,e,",",,m,i,s,s,e,d,,m,y,,p,a,p,i,`,s,,c,a,l,l,s,,f,r,o,m,,t,h,e,,s,w,i,z,z,y,.,,,,,I,`,m,,s,a,d
fixed_text,H,a,d,,a,,c,r,a,z,y,,n,i,g,h,t,",",,l,o,s,t,,k,e,y,s,",",,w,a,l,k,e,d,,h,o,m,e,",",,m,i,s,s,e,d,,m,y,,p,a,p,i,`,s,,c,a,l,l,s,,f,r,o,m,,t,h,e,,s,w,i,z,z,y,.,,I,`,m,,s,a,d,,,


was gonna DM you but it says you`re not following me!    lol  after all I`ve done!  hah


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
raw_text,i,,b,a,s,i,c,a,l,l,y,,w,a,n,t,,t,h,e,,w,o,r,l,d,,t,o,,k,n,o,,t,h,a,t,,i,,h,a,v,e,,t,h,e,,B,E,S,T,,b,e,s,t,,f,r,i,e,n,d,,,,,l,o,v,e,,h,e,r,,<,3
fixed_text,i,,b,a,s,i,c,a,l,l,y,,w,a,n,t,,t,h,e,,w,o,r,l,d,,t,o,,k,n,o,,t,h,a,t,,i,,h,a,v,e,,t,h,e,,B,E,S,T,,b,e,s,t,,f,r,i,e,n,d,,l,o,v,e,,h,e,r,,<,3,,,


hope all is ok


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
char_targets,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
raw_text,,,,,h,o,p,e,,a,l,l,,i,s,,o,k,.
fixed_text,h,o,p,e,,a,l,l,,i,s,,o,k,.,,,,


a riot


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
char_targets,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
raw_text,,,Y,o,u,r,,s,u,c,h,,a,,r,i,o,t,!,!,,,,,Y,o,u,,g,o,,g,i,r,l,!,!
fixed_text,Y,o,u,r,,s,u,c,h,,a,,r,i,o,t,!,!,,Y,o,u,,g,o,,g,i,r,l,!,!,,,,,


In [15]:
import re

In [72]:
re.match(r' [^ ]', ' b   ')

<re.Match object; span=(0, 2), match=' b'>

In [49]:
re.match(r'[apple]', ' a  apple')

In [69]:
for i, row in trn_df.iterrows():
    a = re.match(r' [^ ]', row['text'])
    if a is not None:
        print(f':{row["text"]}:')
        print(a)
        break

: I`d have responded, if I were going:
<re.Match object; span=(0, 2), match=' I'>


In [75]:
re.match(r' apple', ' apple  ')

<re.Match object; span=(0, 6), match=' apple'>

## 

In [3]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [81]:
offsets = tokenizer.encode('apple orange').offsets
offsets

[(0, 5), (5, 12)]

In [82]:
'apple orange'[offsets[0][0]:offsets[0][1]]

'apple'

In [93]:
row = trn_df.query('textID == "3a906c871f"').iloc[0]
row.text.find(row.selected_text)

124

In [95]:
row['selected_text']

'How'

## 自分の処理結果を生成

In [4]:
from tools.datasets import TSEHeadTailDatasetV3

In [5]:
dataset = TSEHeadTailDatasetV3(mode='train', tokenizer_type='roberta_bytelevel_bpe', pretrained_model_name_or_path='../inputs/datasets/', do_lower_case=True, max_length=125, df=trn_df, use_magic=True)

In [93]:
non_magic_dataset = TSEHeadTailDatasetV3(mode='train', tokenizer_type='roberta_bytelevel_bpe', pretrained_model_name_or_path='../inputs/datasets/', do_lower_case=True, max_length=125, df=trn_df, use_magic=False)

In [95]:
reses = []

for i, row in tqdm(list(trn_df.iterrows())):
    reses.append(non_magic_dataset._prep_text(row))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
fixed_trn_df = pd.DataFrame([row.to_dict() for row in reses])
fixed_trn_df.head()

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,...,manual_and_selected_tokenized_intersection_len,fixed_selected_text,mlm_input_ids,mlm_labels,input_ids,attention_mask,token_type_ids,labels_head,labels_tail,offsets
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",...,9,"I`d have responded, if I were going","[0, 7974, 2, 2, 939, 50264, 417, 33, 2334, 6, ...","[-100, -100, -100, -100, -100, 12905, -100, -1...","[0, 7974, 2, 2, 939, 12905, 417, 33, 2334, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,14,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (2, 3..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,...,3,Sooo SAD,"[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 50264...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,7,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 5..."
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,...,2,bullying me,"[0, 2430, 2, 2, 127, 3504, 50264, 11902, 162, ...","[-100, -100, -100, -100, -100, -100, 16, -100,...","[0, 2430, 2, 2, 127, 3504, 16, 11902, 162, 734...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,9,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 8..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,...,3,leave me alone,"[0, 2430, 2, 2, 50264, 1194, 328, 989, 162, 19...","[-100, -100, -100, -100, 99, -100, -100, -100,...","[0, 2430, 2, 2, 99, 1194, 328, 989, 162, 1937,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,10,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 1..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",...,4,"Sons of ****,","[0, 2430, 2, 2, 7250, 9, 31095, 6, 50264, 1705...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 7250, 9, 31095, 6, 596, 1705, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,8,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 8..."


In [96]:
non_magic_fixed_trn_df = pd.DataFrame([row.to_dict() for row in reses])
non_magic_fixed_trn_df.head()

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len,mlm_input_ids,mlm_labels,input_ids,attention_mask,token_type_ids,labels_head,labels_tail,offsets
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",7,9,"[0, 7974, 2, 2, 939, 50264, 417, 33, 2334, 6, ...","[-100, -100, -100, -100, -100, 12905, -100, -1...","[0, 7974, 2, 2, 939, 12905, 417, 33, 2334, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,14,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (2, 3..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,2,3,"[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649,...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,7,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 5..."
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,2,2,"[0, 2430, 2, 2, 127, 3504, 16, 11902, 162, 502...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 127, 3504, 16, 11902, 162, 734...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,9,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 8..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,3,3,"[0, 2430, 2, 2, 99, 1194, 328, 989, 50264, 193...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 99, 1194, 328, 989, 162, 1937,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,10,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 1..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",3,4,"[0, 2430, 2, 2, 7250, 9, 31095, 6, 596, 1705, ...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 7250, 9, 31095, 6, 596, 1705, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,8,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 8..."


In [99]:
diff_cnt = 0

for i in range(len(non_magic_fixed_trn_df)):
    if non_magic_fixed_trn_df.loc[i]['labels_head'] != fixed_trn_df.loc[i]['labels_head']:
        print('===========================')
        print(non_magic_fixed_trn_df.loc[i]['text'])
        print(non_magic_fixed_trn_df.loc[i]['selected_text'])
        print(f"different head: {non_magic_fixed_trn_df.loc[i]['labels_head']} -- {fixed_trn_df.loc[i]['labels_head']}")
        diff_cnt += 1
    elif non_magic_fixed_trn_df.loc[i]['labels_tail'] != fixed_trn_df.loc[i]['labels_tail']:
        print('===========================')
        print(non_magic_fixed_trn_df.loc[i]['text'])
        print(non_magic_fixed_trn_df.loc[i]['selected_text'])
        print(f"different tail: {non_magic_fixed_trn_df.loc[i]['labels_tail']} -- {fixed_trn_df.loc[i]['labels_tail']}")
        diff_cnt += 1
diff_cnt

is back home now      gonna miss every one
onna
different head: 8 -- 9
On the way to Malaysia...no internet access to Twit
.no internet
different head: 11 -- 12
i donbt like to peel prawns, i also dont like going shopping, running out of money and crawling round the car looking for more
dont like go
different tail: 19 -- 18
 which case? I got a new one last week and I`m not thrilled at all with mine.
d I`m not thrilled at all with mine.
different head: 14 -- 15
mannnn..... _ got an iphone!!! im jealous....  http://bit.ly/NgnaR
jealous..
different tail: 18 -- 17
  He`s awesome... Have you worked with him before? He`s a good friend.
s awesome
different head: 6 -- 7
Still no reply from  about my SimFinger problem  So no iRape parody video until I get a response, sorry guys
, sorry guys
different head: 24 -- 25
graduation is done  im a little sad.. anyone want to hang out???
sad.
different tail: 12 -- 11
 hey mia!  totally adore your music.  when will your cd be out?
y adore
different head

 someone came in when I was sleeping off my national passtime and turned me human
d turned me human
different head: 17 -- 18
 I see you didn`t shave your head. But I *LOVE* the cut.  I`m glad you didn`t shave it. Your hair`s too pretty!
m glad
different head: 24 -- 25
 I`ve been unlocked for decades now...just not lucky, never have been.  Gottta make my own luck and that involves $$$$$
.just not lucky, never have been.
different head: 12 -- 13
I must have just missed   & I guess I scared  & _TX off...  Sorry
I scared
different head: 12 -- 13
 Awww. Don`t mean to sound like an overgrown age 37 baby, but I`m still stick. Not as bad as yesterday, but still bad.
t I`m still stick.
different head: 21 -- 22
 there coming this year ^-^  wiL said it himself  but yes now would be good.
ut yes now would be good
different head: 15 -- 16
 oh dear.. u serious? so how does one prevent bites??  I`m so itchy its distracting while editing.. *sigh*
o itchy
different head: 20 -- 21
Omnomnom    JUST came 

 The devil you say!  I`ve been waiting for the Black Widow in movies and now she won`t be awesome at all.
w she won`t be awesome at all
different head: 21 -- 22
 In two months i will have something of the kinf of prom  and i`m really excited! and you haven`t any contact to her?
d i`m really excited
different head: 17 -- 18
thursday night was amazing taylor swift gave me one of her braclets
thursday night was amazing ta
different tail: 11 -- 10
 physics  the most boring class ever!
t boring
different head: 6 -- 7
I hate waiting in lines
I hate wa
different tail: 7 -- 6
 Cheers!  I`d like to thank zBrush for making it all possible
o thank
different head: 10 -- 11
 @_Mintyfresh Thanks very much!!
Thanks very much!
different tail: 13 -- 12
 oh gaha no of course i wasn`t offended why would i be?  i`d love to play for you some day
d love
different head: 22 -- 23
Fighting a migraine   Medication is almost working.
s almost working.
different head: 8 -- 9
 KH is the only thing I`ll be a weeabo

 good tip..... but then my boss would read .... exactly what im supposed to do and would know where I was with the project
good tip.
different tail: 7 -- 6
 oh! Yeah,your mom told be about that.  I thought you meant his twitter ID & I was very confused. That sucks. Sick.
t sucks
different head: 28 -- 29
 aw honey... you just relaxin? That`s probably for the best.
aw honey.
different tail: 7 -- 6
  I`m @ work and missed everythin yesterday but don`t know wot? Anybody
d missed everythin
different head: 9 -- 10
   Sure!  My entire blogroll is terribly updated, but this could motivate me to update and redo it!
uld motiva
different head: 16 -- 17
I`m really nervous about giving a speech at a wedding tomorrow
I`m really nervous ab
different tail: 10 -- 9
 Can you please  with that? I`d love some egg +bacon +ranch +bed for giving birth 3 times in 4 years!
t? I`d love
different head: 8 -- 10
   ...Thank you Mr. President!....
...Thank you Mr. President!.
different tail: 13 -- 12
it`s you and m

Migranes suck....Especially when the kids are suddenly hyper.
suck..
different tail: 9 -- 8
 hahahha but days off are cool when nothings open  its like isolated and haha cool
d haha cool
different head: 21 -- 22
Following _aisa These guys could be shifty but worth following regardless.
but worth following re
different tail: 18 -- 17
Back to work I go....
Back to work I go..
different tail: 10 -- 9
#followfriday     follow these ppl ... they are interesting ..  doesnt tweet much though
l ... they are interesting ..  doesnt tweet much tho
different head: 11 -- 13
 awe thank you!!  good morning to you aswell!!
thank you!
different tail: 8 -- 7
 Ja  Deze is interessanter - Why text messages are limited to 160 characters - http://tr.im/kpgg
s interessanter
different head: 7 -- 8
 ok sweet! and whenever u want, I am stuck in bed all weekend
t, I am stuck in
different head: 10 -- 11
 http://twitpic.com/624qo - but a brown snake bit my duck named Elvis  anyway ahaha Elvis is soo cutee!! And so

...

 dude i am so sorry!!!!! I never got that number for you my fail  i just remembered
dude i am so sorry!
different tail: 10 -- 9
 I like all the songs  I can`t say just one guys! love ya
! love ya
different head: 17 -- 18
iPhone just fell
iPhone just fell
different head: 5 -- 4
Working on several book projects that are due to be released within the next few months.  I need an extra pair of hands!  It`s all good...
l good
different head: 32 -- 33
 yea just fun dive..a bit scary sometimes but fun.  u like diving?
.a bit scary sometimes but fun.
different head: 8 -- 9
LA unified summer schools drastically cut due to CA budget  cuts. Very sad
. Very sad
different head: 15 -- 16
 http://www.CultureShockMag.com  shoutz 2  the mix on the site is gonna b nervvoouuss
na b nervvoouus
different head: 24 -- 25
 The Disney Store is not treating me to well with hours  I realy want to go on 6/13. I willdo anythig to go to the show ;-)
I willdo anythig to go to the show
different head: 27 -- 28
Drinkin

 Then you might enjoy this one too  http://tinyurl.com/ygcp3f
t enjoy
different head: 6 -- 7
  gutted i miss that! the one night i try 2 leave early  lol! friends visitin were tired haha!
re tire
different head: 23 -- 24
 hey! chutti was very tiring.. and have to travel again tomo  how was your Friday?
tiring.
different tail: 14 -- 13
 Well yeah, the hormone things basically a given  thought he had misbehaved specifically to upset your mom and you.
t he had misbehaved specifically to upset your mom and you.
different head: 13 -- 14
Got a Saturdays framed signed foto last night  and a Valentino rossi 1 aswell  eee what a night
e what a night
different head: 26 -- 27
 ....any  sorry wahh!! lub u toooo ;)
.any  sorry wahh!! lub u toooo ;)
different head: 4 -- 5
 Thank you Jon!  That`s very sweet.
s very sweet
different head: 11 -- 12
feels slightly better but **** i sure do miss him.....he`s going to be so hard to get over......  i messed it up ...donuts=comfort
eels slightly better but *

seeing the gf 5 days in a row for a few hours each day is amazing-just wish the location wasn`t a hospital
is amazing-j
different tail: 23 -- 22
mom just woke me u[p and i am so mad i was dreaming about  shoes  she whants me to go to the river it is so stupid
o stupid
different head: 32 -- 33
 I was wondering....   Have a GREAT weekend!
a GREA
different head: 9 -- 10
My besties. If only  was there.  on Twitpic: http://twitpic.com/67hac - My besties. If only  .. http://twitpic.com/67hac
My besties. If only  was there.  on Twitpic: http://twitpic.com/67hac - My besties. If only  .. http://twitpic.com/67ha
different tail: 48 -- 47
Well,  I discovered I can`t swim.  I`m out.  Sorry
.  Sorr
different head: 18 -- 19
 if you like those cookies, you are going to LOVE my cookies   Lars Tetens cookies that is. You coming to see us on 5/16?
o LOVE
different head: 13 -- 14
 Lol...I am greedy. I`m craving Coldstone again.
m greedy.
different head: 7 -- 8
_carter It says 'This video is private.', WT

 didn`t make it to 1000 but I smoked her.  She`s at 60 and I`m at 260.  thank you! How r u?
.  thank yo
different head: 27 -- 28
 I haven`t got ANY cupcakes yet  I was hoping if I said it enough, some would appear!
s hoping
different head: 14 -- 15
 and he REALLY doesn`t like Shiny Happy People. I`m a local REM expert btw. Saw them for the first time in 1986.
Y doesn`t like Shiny Happy People.
different head: 6 -- 7
Jamie @ Sean Cody, up for some angry ****?: Jamie @ Sean Cody, I wouldn`t piss this one off  Hey there Guys, Do.. http://tinyurl.com/ddyyd6
amie @ Sean Cody, up for some angry ****?: Jamie @ Sean Cody, I wouldn`t piss this one off  Hey there Guys, Do.. http://tinyurl.com/ddyyd6
different head: 4 -- 5
The geographY was an exam today!But turned out well  OMG on Wednesday will a English exam xD   I woul`d be very nervous...
ry nervou
different head: 32 -- 33
 -- 5/31 marks the end of our roommate era. And you never made wontons for me again.
1 marks the end of our roommate era

1781

In [100]:
diff_cnt

1781

In [9]:
tkm_fixed_trn_df = pd.read_csv('../inputs/datasets/temp/train_new.csv')
tkm_fixed_trn_df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,don`t force,negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [14]:
trn_df.loc[18].selected_text

'onna'

In [15]:
tkm_fixed_trn_df.loc[18].selected_text

'miss'

In [21]:
fixed_trn_df.loc[18].fixed_selected_text

'miss'

In [11]:
for textID in tqdm(list(fixed_trn_df.textID)):
    my_selected_text = fixed_trn_df[fixed_trn_df.textID == textID].iloc[0]['fixed_selected_text']
    tkm_selected_text = tkm_fixed_trn_df[tkm_fixed_trn_df.textID == textID].iloc[0]['selected_text']
    if my_selected_text != tkm_selected_text:
        print('---------------------------------------')
        print(f'my_selected_text: {my_selected_text}')
        print(f'tkm_selected_text: {tkm_selected_text}')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


---------------------------------------
my_selected_text: 
tkm_selected_text: nan



In [12]:
trn_df[trn_df.selected_text.str.contains('not too ba')]

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
5617,762e9f81f7,My original non-Yahoo acc. was deleted when t...,s not too bad,positive,My original non-Yahoo acc. was deleted when th...,s not too bad,False,"not too bad,",s not too bad,s not too bad,4,4
9832,e898d0d1ab,"good luck! It`s not too bad, and if it is, it...","good luck! It`s not too bad, and if it is, it`...",neutral,"good luck! It`s not too bad, and if it is, it`...","good luck! It`s not too bad, and if it is, it`...",True,"good luck! it`s not too bad, and if it is, it...","good luck! it`s not too bad, and if it is, it...","good luck! it`s not too bad, and if it is, it...",18,21
15345,497470c5f8,I`m not too bad. Just trying to get by so I c...,I`m not too bad. Just trying to get by so I ca...,neutral,I`m not too bad. Just trying to get by so I ca...,I`m not too bad. Just trying to get by so I ca...,True,i`m not too bad. just trying to get by so i c...,i`m not too bad. just trying to get by so i c...,i`m not too bad. just trying to get by so i c...,25,27
17793,3796daa7e7,"oh no!!!! i hope its not too baad, tell mya i...","oh no!!!! i hope its not too baad, tell mya i ...",neutral,"oh no!!!! i hope its not too baad, tell mya i ...","oh no!!!! i hope its not too baad, tell mya i ...",True,"oh no!!!! i hope its not too baad, tell mya i...","oh no!!!! i hope its not too baad, tell mya i...","oh no!!!! i hope its not too baad, tell mya i...",14,20
26010,c7d977c641,That`s not too bad. Good luck!,That`s not too bad. Good luck!,positive,That`s not too bad. Good luck!,That`s not too bad. Good luck!,True,good luck!,that`s not too bad. good luck!,that`s not too bad. good luck!,6,10


In [16]:
import pickle

with open('../inputs/datasets/temp/data2.pkl', 'rb') as fin:
    a = pickle.load(fin)
a

(array([[   0, 7974,  939, ...,    1,    1,    1],
        [   0, 2430,   98, ...,    1,    1,    1],
        [   0, 2430,  127, ...,    1,    1,    1],
        ...,
        [   0, 1313, 1423, ...,    1,    1,    1],
        [   0, 1313,   53, ...,    1,    1,    1],
        [   0, 7974,   70, ...,    1,    1,    1]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]], dt

In [41]:
import json

tkm_df = pd.read_csv('../inputs/origin/train.csv')

with open('../inputs/datasets/temp/data2.pkl', 'rb') as fin:
    res_pkl = pickle.load(fin)

for i, col in enumerate(tqdm(['input_ids', 'attention_mask', 'token_type_ids', 'start_tokens', 'end_tokens'])):
    i_data = res_pkl[i]
    for j, i_data_i in enumerate(tqdm(i_data)):
        tkm_df.loc[j, col] = json.dumps(i_data_i.tolist())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [13]:
fixed_trn_df

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,...,manual_and_selected_tokenized_intersection_len,fixed_selected_text,mlm_input_ids,mlm_labels,input_ids,attention_mask,token_type_ids,labels_head,labels_tail,offsets
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going","I`d have responded, if I were going",True,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going",...,9,"I`d have responded, if I were going","[0, 7974, 2, 2, 939, 50264, 417, 33, 2334, 6, ...","[-100, -100, -100, -100, -100, 12905, -100, -1...","[0, 7974, 2, 2, 939, 12905, 417, 33, 2334, 6, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,14,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 2), (2, 3..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,False,sooo sad,sooo sad,sooo sad,...,3,Sooo SAD,"[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 50264...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 98, 3036, 5074, 939, 40, 2649,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,7,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 5..."
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...,bullying me,False,bullying,bullying me,bullying me,...,2,bullying me,"[0, 2430, 2, 2, 127, 3504, 50264, 11902, 162, ...","[-100, -100, -100, -100, -100, -100, 16, -100,...","[0, 2430, 2, 2, 127, 3504, 16, 11902, 162, 734...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,9,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 3), (3, 8..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,False,leave me alone,leave me alone,leave me alone,...,3,leave me alone,"[0, 2430, 2, 2, 50264, 1194, 328, 989, 162, 19...","[-100, -100, -100, -100, 99, -100, -100, -100,...","[0, 2430, 2, 2, 99, 1194, 328, 989, 162, 1937,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7,10,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 1..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th...","Sons of ****,",False,"sons of ****,","sons of ****,","sons of ****,",...,4,"Sons of ****,","[0, 2430, 2, 2, 7250, 9, 31095, 6, 50264, 1705...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 7250, 9, 31095, 6, 596, 1705, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,8,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27418,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish we could come see u on Denver husband los...,d lost,False,lost his job and can`t afford it,husband lost,d lost,...,1,lost,"[0, 2430, 2, 2, 2813, 52, 115, 283, 192, 1717,...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 2813, 52, 115, 283, 192, 1717,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",14,15,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 5), (5, 8..."
27419,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,I`ve wondered about rake to. The client has ma...,", don`t force",False,don`t force,", don`t force",", don`t force",...,5,don`t force,"[0, 2430, 2, 2, 939, 12905, 548, 12267, 59, 37...","[-100, -100, -100, -100, -100, -100, -100, -10...","[0, 2430, 2, 2, 939, 12905, 548, 12267, 59, 37...","[1, 1, 1, 1, 1, 1,

In [57]:
tkm_df['labels_head'] = tkm_df.start_tokens.apply(lambda x: np.argmax(json.loads(x)) + 2)
tkm_df['labels_tail'] = tkm_df.end_tokens.apply(lambda x: np.argmax(json.loads(x)) + 2 + 1)

In [59]:
for textID in tqdm(list(fixed_trn_df.textID)):
    my_start_label = fixed_trn_df[fixed_trn_df.textID == textID].iloc[0]['labels_head']
    my_end_label = fixed_trn_df[fixed_trn_df.textID == textID].iloc[0]['labels_tail']
    tkm_start_label = tkm_df[tkm_df.textID == textID].iloc[0]['labels_head']
    tkm_end_label = tkm_df[tkm_df.textID == textID].iloc[0]['labels_tail']
    if my_start_label != tkm_start_label or my_end_label != tkm_end_label:
        print('---------------------------------------')
        print(textID)
        print(f'my_start_label: {my_start_label} -- tkm_start_label: {tkm_start_label}')
        print(f'my_end_label: {my_end_label} -- tkm_end_label: {tkm_end_label}')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


---------------------------------------
780c673bca
my_start_label: 4 -- tkm_start_label: 2
my_end_label: 5 -- tkm_end_label: 3
---------------------------------------
12f21c8f19
my_start_label: 4 -- tkm_start_label: 2
my_end_label: 5 -- tkm_end_label: 3
---------------------------------------
96ff964db0
my_start_label: 14 -- tkm_start_label: 2
my_end_label: 15 -- tkm_end_label: 3
---------------------------------------
eaf2942ee8
my_start_label: 6 -- tkm_start_label: 2
my_end_label: 7 -- tkm_end_label: 3
---------------------------------------
4c279acff6
my_start_label: 7 -- tkm_start_label: 2
my_end_label: 8 -- tkm_end_label: 3
---------------------------------------
3a906c871f
my_start_label: 32 -- tkm_start_label: 2
my_end_label: 33 -- tkm_end_label: 3
---------------------------------------
09d0f8f088
my_start_label: 4 -- tkm_start_label: 2
my_end_label: 5 -- tkm_end_label: 3



In [85]:
trn_df.query('textID == "96ff964db0"')

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
11807,96ff964db0,"4 hours of sleep, a migraine, again? What is w...",hat,negative,"4 hours of sleep, a migraine, again? What is w...",hat,False,hate my life,what,hat,0,0


In [70]:
row = fixed_trn_df.sample(1, random_state=81).iloc[0]
print(row['text'])
print(' ' + ' '.join(row['text'].split()))
for (offset1, offset2) in row['offsets']:
    print(' -------------------- ')
    print(offset1, offset2)
    print(f":{(' ' + ' '.join(row['text'].split()))[offset1:offset2]}:")

Watch Jay Leno tonite!!!  It`s his last day as the Tonight Show Host
 Watch Jay Leno tonite!!! It`s his last day as the Tonight Show Host
 -------------------- 
0 0
::
 -------------------- 
0 0
::
 -------------------- 
0 0
::
 -------------------- 
0 0
::
 -------------------- 
0 6
: Watch:
 -------------------- 
6 8
: J:
 -------------------- 
8 10
:ay:
 -------------------- 
10 14
: Len:
 -------------------- 
14 15
:o:
 -------------------- 
15 19
: ton:
 -------------------- 
19 22
:ite:
 -------------------- 
22 25
:!!!:
 -------------------- 
25 28
: It:
 -------------------- 
28 29
:`:
 -------------------- 
29 30
:s:
 -------------------- 
30 34
: his:
 -------------------- 
34 39
: last:
 -------------------- 
39 43
: day:
 -------------------- 
43 46
: as:
 -------------------- 
46 50
: the:
 -------------------- 
50 58
: Tonight:
 -------------------- 
58 63
: Show:
 -------------------- 
63 68
: Host:
 -------------------- 
0 0
::
 -------------------- 
0 0
::
 ----------

In [60]:
fixed_trn_df.query('textID == "780c673bca"')

,textID,text,selected_text,sentiment,my_text,my_selected_text,my_text_eq_my_selected_text,predicted_texts,manual_selected_text,selected_text_lower,...,manual_and_selected_tokenized_intersection_len,fixed_selected_text,mlm_input_ids,mlm_labels,input_ids,attention_mask,token_type_ids,labels_head,labels_tail,offsets
5423,780c673bca,going out for the good ol` `soak` tonight for ...,ing,positive,going out for the good ol` `soak` tonight for ...,ing,False,in love,going,ing,...,0,ing,"[0, 1313, 2, 2, 164, 50264, 13, 50264, 205, 22...","[-100, -100, -100, -100, -100, 66, -100, 5, -1...","[0, 1313, 2, 2, 164, 66, 13, 5, 205, 22796, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,5,"[(0, 0), (0, 0), (0, 0), (0, 0), (0, 6), (6, 1..."


In [56]:
a = tkm_df.query('textID == "cb774db0d1"')
a

,textID,text,selected_text,sentiment,input_ids,attention_mask,token_type_ids,start_tokens,end_tokens,labels_head,labels_tail
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[0, 7974, 939, 12905, 417, 33, 2334, 6, 114, 9...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",4,13


In [51]:
np.argmax(a.iloc[0]['end_tokens'])

0

In [52]:
a.iloc[0]['end_tokens']

'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [82]:
for i, row in fixed_trn_df.query('sentiment != "neutral"').sample(100, random_state=86).iterrows():
    start = row['labels_head']
    end = row['labels_tail']
    start_offset = row['offsets'][start][0]
    end_offset = row['offsets'][end - 1][1]
    text = ' ' + ' '.join(row['text'].split())
    print('----------------------------')
    print('  ' in row['text'][:(start+end) // 2])
    print(row['selected_text'])
    print(text[start_offset:end_offset])

----------------------------
False
thankyooooou
 thankyooooou
----------------------------
False
JONAS BROTHERS - Live to party.                It`s rocking so hard
 JONAS BROTHERS - Live to party. It`s rocking so hard
----------------------------
False
love
 love
----------------------------
False
Happy
 Happy
----------------------------
False
sick
 sick
----------------------------
False
I won`t whine about bad beats  ..
 I won`t whine about bad beats ..
----------------------------
True
Ya, it was pretty bad
 Ya, it was pretty bad
----------------------------
False
New forest hoping the walk would lose me some pounds
 New forest hoping the walk would lose me some pounds
----------------------------
False
you don`t do sympathy very well
 you don`t do sympathy very well
----------------------------
False
Glorious Su
 Glorious Sund
----------------------------
False
SO funny..
 SO funny...
----------------------------
False
Yes, it is nice.
 Yes, it is nice.
--------------------------

## check original dataset

In [127]:
def html_excape(text):
    from_ = ["&amp;", "&lt;", "&gt;", "&quot;", "&#39;"]
    to =  ["&", "<", ">" ,'"', "'"]
    for i in range(4):
        text = text.replace(from_[i], to[i])
    return text.replace("'", '`').replace('"', "'")

In [148]:
def fix_target(text: str, selected_text: str, sentiment: str):
    text_fixed, selected_text_fixed = text, selected_text
    ss = text.find(selected_text)
    if text[max(ss - 2, 0):ss] == '  ':
        ss -= 2
    if ss > 0  and text[ss - 1] == ' ':
        ss -= 1
    ee = ss + len(selected_text)

    if re.match(r' [^ ]', text) is not None:
        ee -= 1
    ss = max(0, ss)
    if '  ' in text[:ss] and sentiment != 'neutral':
        text_fixed = " ".join(text.split())
        selected_text_fixed = text_fixed[ss:ee].strip()
        if len(selected_text_fixed) > 1 and selected_text_fixed[-2] == ' ':
            selected_text_fixed = selected_text_fixed[:-2]

    maxlen = len(text)
    fixed_text_padded = text_fixed + ' ' * (maxlen - len(text_fixed))
    return pd.Series(dict(
        text_fixed=text_fixed,
        text_fixed_padded=fixed_text_padded,
        selected_text_fixed=selected_text_fixed,
    ))

In [152]:
origin_df = pd.read_csv('../inputs/datasets/w_private/complete-tweet-sentiment-extraction-data/tweet_dataset.csv').dropna()
display(origin_df.shape, origin_df.head())

(27767, 8)

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging @ghostridah14: why didn't you go to...,a610d6b25b,negative,didn`t like my


In [153]:
origin_df.textID.isnull().mean()

0.0

In [239]:
origin_df['is_contain_ng_word'] = (origin_df.text.fillna('').str.contains('[^\*]\*\*\*\*[^\*]')) | (origin_df.text.fillna('').str.contains('[^\*]\*\*\*\*$')) | (origin_df.text.fillna('').str.contains('^\*\*\*\*[^\*]'))
origin_df['is_contain_at'] = origin_df.text.fillna('').str.contains('@')
origin_df['processed_old_text'] = origin_df.old_text.apply(html_excape).apply(lambda x: re.sub(r'@[a-zA-Z0-9]+',  '', ' '.join(x.split())))
display(origin_df.shape, origin_df.head())

(27767, 14)

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,is_contain_ng_word,is_contain_at,processed_old_text,text_fixed,text_fixed_padded,selected_text_fixed
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache,False,False,Layin n bed with a headache ughhhh...waitin on...,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy,False,False,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!,False,False,wants to hang out with friends SOON!,wants to hang out with friends SOON!,wants to hang out with friends SOON!,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...,False,False,We want to trade with someone who has Houston...,We want to trade with someone who has Houston...,We want to trade with someone who has Houston...,We want to trade with someone who has Houston ...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging @ghostridah14: why didn't you go to...,a610d6b25b,negative,didn`t like my,False,False,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging : why didn`t you go to prom? BC my ...,didn`t like my


In [156]:
fixed_df = origin_df.progress_apply(
    lambda x: fix_target(x.text, x.selected_text, x.sentiment),
    axis=1
)
origin_df = origin_df.assign(**fixed_df)

In [157]:
for i, row in origin_df.query('is_contain_ng_word and is_contain_at').sample(10, random_state=71).iterrows():
    with pd.option_context('display.max_columns', None):
        max_len = max(len(row.text), len(row.processed_old_text))
        df = pd.DataFrame(dict(
            text=list(row.text) + [' '] * (max_len - len(row.text)),
            processed_old_text=list(row.processed_old_text) + [' '] * (max_len - len(row.processed_old_text)),
        )).T
        display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120
text,@,_,N,e,w,A,m,e,r,y,k,a,h,_,,i,m,,a,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,,a,b,o,u,t,,t,h,a,t,.,.,.,,a,,*,*,*,*,,w,o,k,e,,u,p,,l,a,t,e,.,.,.,,d,i,d,n,t,,g,e,t,,m,y,,b,r,e,a,k,f,a,s,t,,(,w,a,k,e,n,b,a,k,e,),
processed_old_text,@,_,N,e,w,A,m,e,r,y,k,a,h,_,,i,m,,a,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,l,,a,b,o,u,t,,t,h,a,t,.,.,.,,a,,n,i,g,g,a,,w,o,k,e,,u,p,,l,a,t,e,.,.,.,,d,i,d,n,t,,g,e,t,,m,y,,b,r,e,a,k,f,a,s,t,,(,w,a,k,e,n,b,a,k,e,)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
text,S,o,,a,n,n,o,y,e,d,,I,,m,i,s,s,e,d,,M,i,t,c,h,,B,e,n,n,,@,,L,e,m,o,n,,T,r,e,e,.,,*,*,*,*,,y,o,u,,W,h,a,t,`,s,,O,n,!
processed_old_text,S,o,,a,n,n,o,y,e,d,,I,,m,i,s,s,e,d,,M,i,t,c,h,,B,e,n,n,,@,,L,e,m,o,n,,T,r,e,e,.,,d,a,m,n,,y,o,u,,W,h,a,t,`,s,,O,n,!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46
text,@,,w,o,r,k,,d,o,,n,o,t,,f,e,e,l,,l,i,k,e,,b,e,i,n,,i,n,,d,i,s,,*,*,*,*,,t,o,d,a,y,
processed_old_text,@,,w,o,r,k,,d,o,,n,o,t,,f,e,e,l,,l,i,k,e,,b,e,i,n,,i,n,,d,i,s,,b,i,t,c,h,,t,o,d,a,y


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
text,@,_,h,a,y,l,e,s,,a,r,e,,u,,s,e,r,i,o,u,s,?,?,?,?,,y,o,u,,t,o,u,c,h,e,d,,h,i,s,,*,*,*,*,?,,o,h,m,y,g,o,d,,u,,*,*,*,*,!,!,!,,l,o,l,,a,a,a,a,a,w,w,w,w,w,w,w,,y,,d,i,d,,u,,t,e,l,l,,m,e,,t,h,a,t,!,!,!,
processed_old_text,@,_,h,a,y,l,e,s,,a,r,e,,u,,s,e,r,i,o,u,s,?,?,?,?,,y,o,u,,t,o,u,c,h,e,d,,h,i,s,,a,r,s,e,?,,o,h,m,y,g,o,d,,u,,b,i,t,c,h,!,!,!,,l,o,l,,a,a,a,a,a,w,w,w,w,w,w,w,,y,,d,i,d,,u,,t,e,l,l,,m,e,,t,h,a,t,!,!,!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135
text,@,_,A,i,s,l,i,n,n,T,i,g,h,e,e,,b,h,a,h,a,",",,i,t,s,,a,,t,e,e,n,a,g,e,,n,i,g,h,t,c,l,u,b,,a,n,d,,i,,a,m,,a,t,,h,o,m,e,,w,h,e,n,,i,m,,s,u,p,p,o,s,e,d,,t,o,,b,e,,t,h,e,r,e,.,,t,h,e,y,,h,a,d,,n,o,,*,*,*,*,,l,i,c,e,n,s,e,.,,n,o,,m,o,r,e,,t,o,u,c,h,,,
processed_old_text,@,_,A,i,s,l,i,n,n,T,i,g,h,e,e,,b,h,a,h,a,",",,i,t,s,,a,,t,e,e,n,a,g,e,,n,i,g,h,t,c,l,u,b,,a,n,d,,i,,a,m,,a,t,,h,o,m,e,,w,h,e,n,,i,m,,s,u,p,p,o,s,e,d,,t,o,,b,e,,t,h,e,r,e,.,,t,h,e,y,,h,a,d,,n,o,,f,u,c,k,i,n,g,,l,i,c,e,n,s,e,.,,n,o,,m,o,r,e,,t,o,u,c,h


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93
text,M,y,,n,o,s,e,,i,s,,r,u,n,n,y,",",,m,y,,h,e,a,d,,i,s,,p,o,u,n,d,i,n,g,",",,a,n,d,,t,e,e,t,h,,h,u,r,t,,l,i,k,e,,a,,*,*,*,*,.,,M,A,N,.,,@,-,),,I,,f,e,e,l,,a,w,f,u,l,.,,,=,(,(
processed_old_text,M,y,,n,o,s,e,,i,s,,r,u,n,n,y,",",,m,y,,h,e,a,d,,i,s,,p,o,u,n,d,i,n,g,",",,a,n,d,,t,e,e,t,h,,h,u,r,t,,l,i,k,e,,a,,b,i,t,c,h,.,,M,A,N,.,,@,-,),,I,,f,e,e,l,,a,w,f,u,l,.,,=,(,(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
text,_,d,a,_,*,*,*,*,,ï,¿,½,,c,a,n,t,,i,m,,n,o,t,,@,,m,y,,s,i,s,t,e,r,,h,o,u,s,e,,n,o,,ï,¿,½,-,n,e,t,,t,o,n,i,t,e,,,
processed_old_text,_,d,a,_,a,s,s,h,o,l,e,,ï,¿,½,,c,a,n,t,,i,m,,n,o,t,,@,,m,y,,s,i,s,t,e,r,,h,o,u,s,e,,n,o,,ï,¿,½,-,n,e,t,,t,o,n,i,t,e


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
text,f,e,e,l,s,,l,i,k,e,,*,*,*,*,,w,h,e,n,e,v,e,r,,s,h,e,,l,a,y,s,,d,o,w,n,.,,@,),(,1,+,-,`,$,#,!,,I,,h,a,t,e,,h,e,a,r,t,b,u,r,n,,&,,b,a,c,k,a,c,h,e,s,!
processed_old_text,f,e,e,l,s,,l,i,k,e,,c,r,a,p,,w,h,e,n,e,v,e,r,,s,h,e,,l,a,y,s,,d,o,w,n,.,,@,),(,1,+,-,`,$,#,!,,I,,h,a,t,e,,h,e,a,r,t,b,u,r,n,,&,,b,a,c,k,a,c,h,e,s,!


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
text,,L,M,A,O,!,I,,w,a,s,,J,U,S,T,,t,h,i,n,k,i,n,g,,h,o,w,,I,,*,*,*,*,,h,a,t,e,,@,,l,e,a,s,t,,3,,p,e,o,p,l,e,,t,h,a,t,,I,`,m,,f,o,l,l,o,w,i,n,g,.,.,.,b,u,t,,I,,h,a,v,e,n,`,t,,u,n,f,o,l,l,o,w,e,d,,e,m,`,,y,e,t,.,,I,`,m,,a,,l,o,o,n,.,,,
processed_old_text,,L,M,A,O,!,I,,w,a,s,,J,U,S,T,,t,h,i,n,k,i,n,g,,h,o,w,,I,,f,u,c,k,i,n,g,,h,a,t,e,,@,,l,e,a,s,t,,3,,p,e,o,p,l,e,,t,h,a,t,,I,`,m,,f,o,l,l,o,w,i,n,g,.,.,.,b,u,t,,I,,h,a,v,e,n,`,t,,u,n,f,o,l,l,o,w,e,d,,e,m,`,,y,e,t,.,,I,`,m,,a,,l,o,o,n,.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
text,@,,t,h,e,,d,r,s,.,,i,n,,b,e,v,e,r,l,y,,h,i,l,l,s,.,.,.,d,o,c,t,o,r,`,s,,s,c,a,r,e,,*,*,*,*,,o,u,t,,o,f,,m,e
processed_old_text,@,,t,h,e,,d,r,s,.,,i,n,,b,e,v,e,r,l,y,,h,i,l,l,s,.,.,.,d,o,c,t,o,r,`,s,,s,c,a,r,e,,c,r,a,p,,o,u,t,,o,f,,m,e


In [ ]:
ng_word_list

In [198]:
origin_df.query('is_contain_ng_word').text.apply(lambda x: x.count('****')).value_counts()

1    935
2     59
3      5
4      3
Name: text, dtype: int64

In [ ]:
['crap', 'fuck', 'asshole', 'nigga', 'damn', 'sucks', 'shit', 'cum', 'shitty',
  't it', 'ass', 'bitch', 'xxx', 'xx', 'crotch', 'tits', 'mofo', 'whore', 'cr*p',
  'pussy', 'gay', 'porn', 'fuk', 'fcuk', 'strip', 'bastard', 'twat', 'sex',
  'dick', 'arse', 'f*ck', 'penis', 'rape', 'boobs', 'nipple', 'sh*t', 'x_x',
  'nude', 'masturbate', 'no b', 'busty', 'cock', 'smut', 'cunts', 'hooker',
  'f-u-c-k', 't, it', 'horny', 'x x', 'skank', 'anal', 'gangbang', 'slut', 'a 55',
  'butthole', 'panties', 's ex', 'jizz', 'tit-boob', '@$$', 'cowgirls', 'boob', 'x.x',
  'ti?t', 'playboy', ]

In [289]:
ng_words = ['crap', 'fuck', 'asshole', 'nigga', 'damn', 'sucks', 'shit', 'cum', 'shitty',
                       't it', 'ass', 'bitch', 'xxx', 'xx', 'crotch', 'tits', 'mofo', 'whore', 'cr*p',
                       'pussy', 'gay', 'porn', 'fuk', 'fcuk', 'strip', 'bastard', 'twat', 'sex',
                       'dick', 'arse', 'f*ck', 'penis', 'rape', 'boobs', 'nipple', 'sh*t', 'x_x',
                       'nude', 'masturbate', 'no b', 'busty', 'cock', 'smut', 'cunts', 'hooker',
                       'f-u-c-k', 't, it', 'horny', 'x x', 'skank', 'anal', 'gangbang', 'slut', 'a 55',
                       'butthole', 'panties', 's ex', 'jizz', 'tit-boob', '@$$', 'cowgirls', 'boob', 'x.x',
                       'ti?t', 'playboy', ]

for i, row in tqdm(list(origin_df.query('is_contain_ng_word').iterrows())):
    idx = row['text'].find('****')
    for ng_word in ng_words:
        # if ng_word in row['processed_old_text'][idx:]:
        if ng_word in row['processed_old_text'].lower():
            break
        else:
            continue
    else:
        if row['text'] == ' thanks for reading my blog. and for laughing at MY PERILS AND TRAGEDIES! ****Cries****':
            continue
        if row['text'] == 'BOOK NOW & SAVE:SUMMER 2009 * THE AMAZONES VILLAGE SUITES****-CRETE-GREECE! THE BEST PLACE TO BE!':
            continue
        print(row['text'])
        print(row['processed_old_text'])
        break

In [139]:
origin_df.query('is_contain_ng_word').text.apply(lambda x: x.count('****')).value_counts()

1    1357
0     438
2      87
3       6
4       4
Name: text, dtype: int64

In [139]:
origin_df.query('is_contain_ng_word').text.apply(lambda x: x.count('****')).value_counts()

1    1357
0     438
2      87
3       6
4       4
Name: text, dtype: int64

In [138]:
'******'.count('**')

3